In [31]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script loads the results from wp_all_8_current_density_metrics, which are 3
# .csv files, and correlates current density metrics with DCM values.
# The analysis method used is Canonical correlation analysis (CCA)
# Add details on CCA
## WORKING ON THIS AT THE MOMENT (25012022)
# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
import scipy.stats

In [32]:
## Parameters and variables: 
results_folder = 'D:\\roast-chapter3\\wp_all_results\\' # Folder with results (current density)
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' # Project folder
dcm_results_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\all_dcm_results\\' # Folder with dcm results

# Datasets names and subjects lists
# db_names = ['wp2a', 'wp1b'] # waiting for wp1a results from Davinia
db_names = ['wp2a']

In [45]:
from sklearn.cross_decomposition import CCA

for db_id, db in enumerate(db_names):

    # Loading DCM results (nb: the unthresholded ones, for the correlation analysis)
    DCM = np.load(os.path.join(dcm_results_folder, db + '_dcm_unthresholded.npy'))
    DCM_sham = np.load(os.path.join(dcm_results_folder, db + '_dcm_sham_unthresholded.npy'))
    # Loading current density results
    cd_metrics = pd.read_csv(os.path.join(results_folder, db +'_current_density_results.csv'))
    cd_metrics.drop('Unnamed: 0', axis = 1, inplace = True) # Dropping participants' id

    # Simple R correlation (example)
    #x = cd_metrics['m1_medians'].values # Medians current density in M1
    #y = DCM[:, 0, 0] # NB: DCM index related to m1 <-> m1 (of all subjects) = [0,0]
    #r,p  = scipy.stats.pearsonr(x, y)    # Pearson's r
    #print ('R: ' + str(round(r,3)) + ', p: ' +  str(round(p,3)))

    # Extracting DCM values I am interested in (m1<->m1, th<->th, m1->th, th->m1)
    m1m1 = DCM[:,0,0]
    thth = DCM[:,1,1]
    m1th = DCM[:,1,0]
    thm1 = DCM[:,0,1]
    dcm_df = pd.DataFrame(zip(m1m1, thth, m1th, thm1),
                                columns = ['m1m1', 'thth', 'm1th',
                                'thm1'])

    # We first standardise our variables by subtracting with mean and dividing by standard deviation
    cd_df_s = (cd_metrics-cd_metrics.mean())/(cd_metrics.std())
    dcm_df_s = (dcm_df-dcm_df.mean())/(dcm_df.std())

    print(cd_df_s.head())
    print(dcm_df_s.head())

    ''' Damian: Basically your CCA analysis will give you pairs of canonical variates (one of
    each pair that describes the current measures, and one that describes the DCM). For each
    significant pair, you then correlate the CV from one side with each of the individual
    measures from the other side.
    This will basically tell you that e.g., Current density overall is most strongly related to
    DCM connectivity in pathways X and Y, and that DCM connectivity overall is most strongly
    related to current desnity in region A and B. '''

    cca = CCA(n_components = 2)
    cca.fit(cd_df_s, dcm_df_s)
    X_c, Y_c = cca.transform(cd_df_s, dcm_df_s) # X = current measures, Y = DCM measures    

    cc_res = pd.DataFrame({"CCX_1":X_c[:, 0],
                       "CCY_1":Y_c[:, 0],
                       "CCX_2":X_c[:, 1],
                       "CCY_2":Y_c[:, 1]})


   m1_medians    m1_max  th_medians    th_max  cb_medians    cb_max
0    1.064099 -0.222754   -0.387600  2.239864   -0.733822 -0.615235
1    1.202965  1.111482   -0.967031 -0.837454   -0.063144 -0.028971
2   -0.107853  1.544385   -0.503373  0.143153   -0.851981 -1.144642
3   -0.665023 -1.268581   -0.250689 -0.461470    0.746472 -1.155159
4    0.777813  0.006170    1.643100 -0.504230    2.059322  0.327670
       m1m1      thth      m1th      thm1
0  0.300931  1.085120 -1.125615 -0.815612
1 -1.533606  0.598047  0.172432 -0.076831
2 -2.011669 -1.517923 -0.709467  0.424610
3  1.769832  0.171052 -0.640999 -0.924352
4 -0.010210  0.366904 -0.165926 -0.092548


In [48]:
cc_res




,CCX_1,CCY_1,CCX_2,CCY_2
0,-0.106573,0.628324,1.026583,0.629243
1,-0.942991,-1.002685,-0.182874,0.469841
2,-0.942674,-2.611430,0.558471,-0.068010
3,0.544622,1.580208,-0.109502,-0.439223
4,1.019010,0.137248,1.234024,0.246762
5,0.457479,0.231174,-0.749356,0.149859
6,1.654695,2.300914,-0.506708,-0.416244
7,-0.250775,-1.952809,2.820099,0.818899
8,-0.331092,1.265603,0.904636,0.143104
9,-0.301389,-1.375746,-1.479075,-1.146511
